In [1]:
import os 
packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [7]:
df1 = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:9092")\
.option("subscribe", "Trump")\
.load()

In [8]:
query2 = df1.writeStream\
    .queryName("device_counts").format("memory")\
    .start()

IllegalArgumentException: Cannot start query with name device_counts as a query with that name is already active in this SparkSession

In [17]:
spark.sql('select count(*) FROM device_counts').show()
spark.sql('SELECT * FROM device_counts').show()

+--------+
|count(1)|
+--------+
|    6666|
+--------+

+----+-------------+-----+---------+------+--------------------+-------------+
| key|        value|topic|partition|offset|           timestamp|timestampType|
+----+-------------+-----+---------+------+--------------------+-------------+
|null|[64 61 74 61]|Trump|        0|   973|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   974|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   975|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   976|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   977|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   978|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   979|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   980|2020-12-13 08:38:...|            0|
|null|[64 61 74 61]|Trump|        0|   981|2020-12-13 08:38:...|           

In [10]:
df1.writeStream.trigger(processingTime='5 seconds')\
.format("console").outputMode("complete").start()

AnalysisException: Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;;
kafka